In [25]:
import numpy as np
import matplotlib.pyplot as plt

import pygame, sys
from pygame.locals import *
from matplotlib.backends.backend_agg import FigureCanvasAgg
from matplotlib.figure import Figure

In [26]:
def make_surface_rgba(canvas):
    array               = np.swapaxes(np.asarray(canvas.buffer_rgba()), 0, 1)
    surface             = pygame.Surface(array.shape[0:2], pygame.SRCALPHA, 32)
    pygame.pixelcopy.array_to_surface(surface, array[:, :, 0:3])
    surface_alpha       = np.array(surface.get_view('A'), copy=False)
    surface_alpha[:, :] = array[:, :, 3]

    return surface

def drawPlot(screen, pos, X, Y, X2, Y2, XBest):
    fig    = Figure(figsize=(7.5, 5), dpi=100, constrained_layout=True)
    canvas = FigureCanvasAgg(fig)
    plt1   = fig.add_subplot(2, 1, 1)
    plt1.plot(X, Y, linewidth=2, color='r')
    plt2   = fig.add_subplot(2, 1, 2)
    plt2.plot(X, Y, linewidth=2, color='b')
    plt2.scatter(X2, Y2, linewidth=2, color='k')
    plt2.axvline(x=XBest)
    canvas.draw()
    screen.blit(make_surface_rgba(canvas), pos)
    
def drawPlot3d(screen, pos, x_2d, y_2d, z_2d, points):
    fig    = Figure(figsize=(7.5, 5), dpi=100, constrained_layout=True)
    canvas = FigureCanvasAgg(fig)
    ax     = fig.add_subplot(projection='3d')
    surf   = ax.plot_surface(x_2d, y_2d, z_2d, cmap='viridis', linewidth=0)
    surf   = ax.scatter(points[:, 0], points[:, 1], points[:, 2], linewidth=2, color='k')
    canvas.draw()
    screen.blit(make_surface_rgba(canvas), pos)
    
def saveFitnessPlot(fitnessPlot, file):
    fig    = Figure(figsize=(7.5, 5), dpi=100, constrained_layout=True)
    canvas = FigureCanvasAgg(fig)
    plt1   = fig.add_subplot(1, 1, 1)
    XY     = np.array([[v[0], v[1]] for v in enumerate(fitnessPlot)])
    plt1.plot(XY[:, 0], XY[:, 1], linewidth=2, color='r')
    canvas.draw()
    pygame.image.save(make_surface_rgba(canvas), file)

pygame.font.init()
my_font = pygame.font.SysFont('Arial MS', 18)

def drawText(screen, text, pos):
    text_surface = my_font.render(text, False, (0, 0, 0))
    screen.blit(text_surface, pos) 

In [27]:
def rand(a, b):
    return a + (b - a) * np.random.rand()

def randVec(A, B):
     return np.array([rand(A[i], B[i]) for i in range(len(A))])

def limit(X, lims):
     return np.array([min(lims[i][1], max(lims[i][0], X[i])) for i in range(len(X))])
    
class Solution:
    def __init__(self, X):
        self.X = X

    def Mutation(self, km, ks, lims):
        if rand(0, 1) < ks:
            X = randVec(lims[:, 0], lims[:, 1])
        else:
            X = limit(self.X + rand(-km, km), lims)

        return Solution(X)

In [28]:
class GA:
    def __init__(self, N, lims):
        self.N     = N
        self.lims  = np.array(lims)
        self.nDims = len(self.lims)
        
        if self.lims.ndim == 1:
            self.lims = self.lims.reshape(1, self.nDims)

        self.population = []

        for i in range(N):
            R = randVec(self.lims[:, 0], self.lims[:, 1])
            self.population.append(Solution(R))

        self.firstTime = True
    
    def CalcFitness(self, F, A, B):
        for i in range(len(self.population)):
            s   = self.population[i]
            s.f = F(*s.X, A, B)

    def SelectBest(self, num):
        best_population = sorted(self.population, key=lambda s:s.f, reverse=True)
        
        return best_population[:num]

    def Crossing(self, A, B):
        a = A.X
        b = B.X
        k = np.random.rand()
        C = Solution(limit(k * a + (1 - k) * b, self.lims))
        k = np.random.rand()
        D = Solution(limit(k * a + (1 - k) * b, self.lims))
        
        return C, D
    
    def Iteration(self, F, A, B, ks):
        if self.firstTime:
            self.CalcFitness(F, A, B)
            self.firstTime = False
        
        num_elite       = 3
        self.population = self.SelectBest(num_elite)

        while len(self.population) < self.N:
            i    = np.random.randint(0, num_elite - 2)
            j    = np.random.randint(i, num_elite - 1)
            C, D = self.Crossing(self.population[i], self.population[j])
            C    = C.Mutation(0.5, ks, self.lims)
            D    = D.Mutation(0.5, ks, self.lims)

            if len(self.population) < self.N:
                self.population.append(C)

            if len(self.population) < self.N:
                self.population.append(D)

        self.CalcFitness(F, A, B)
        self.population = sorted(self.population, key=lambda s: s.f, reverse=True)
        
    def GetBest(self):
        return self.population[0]

In [29]:
A, B = 0, -2

def F(x, A, B):
    return 1 / (1 + x) + A * np.sin(x**2 / 25) - B * np.cos(x**3 / 45)

def F2(x, y, A, B):
    return 0.1 * (x ** 2 + y ** 2 + A * x + B * y) + np.exp(np.cos(1.5 * x)**2 - np.sin(1.2 * y)**2) - 0.8
    
x          = np.linspace(-3, 3, 100)
y          = np.linspace(-3, 3, 100)
x_2d, y_2d = np.meshgrid(x, y)
z_2d       = F2(x_2d, y_2d, A, B)

X = np.array(np.arange(0, 10, 0.1))
Y = [F(x, A, B) for x in X]

def main(N=10, ks=0.1, ftype=0):
    W           = 800
    H           = 600
    screen      = pygame.display.set_mode((W, H))
    timer       = pygame.time.Clock()
    fps         = 10
    iteration   = 0
    
    if not ftype:
        ga      = GA(N, ([0, 10]))
    else:
        ga      = GA(N, ([-3,3], [-3,3]))

    
    fitnessPlot = []
    
    while True:
        for event in pygame.event.get():
            if event.type == KEYDOWN:
                if event.key == pygame.K_i:
                    ga.Iteration(F if not ftype else F2, A, B, ks)
                    iteration += 1
                    fitnessPlot.append(ga.GetBest().f)

                if event.key == pygame.K_s:
                    saveFitnessPlot(fitnessPlot, "fitness.png")
                
                if event.key == pygame.K_x:
                    pygame.quit()
                    return
        
        if not ftype:
            X2    = [s.X for s in ga.population]
            Y2    = [F(X[0], A, B) for X in X2]
            XBest = ga.GetBest().X[0]

            screen.fill((255, 255, 255))
            drawPlot(screen, (0, 0), X, Y, X2, Y2, XBest)
            drawText(screen, f"Iter: {iteration}", (45, H - 75))
            drawText(screen, f"Keys: i - Iter; s - Save; x - Exit", (45, H - 50))
        else:
            points = np.array([[s.X[0], s.X[1], F2(s.X[0], s.X[1], A, B)] for s in ga.population])
            XBest  = ga.GetBest().X
            ZBest  = F2(XBest[0], XBest[1], A, B)

            screen.fill((255,255,255))
            drawPlot3d(screen, (0, 0), x_2d, y_2d, z_2d, points)
            drawText(screen, f"Iter: {iteration}, X={XBest[0]:.2f}, Y={XBest[1]:.2f}, Z={ZBest:.2f}", (45, H - 75))
            drawText(screen, f"Keys: i - Iter; s - Save; x - Exit", (45, H - 50))
        
        pygame.display.flip()
        timer.tick(fps)

In [30]:
main(N=20, ks=0.5, ftype=1)